In [1]:
#Notebook to re-run the local and remote perturbation cases
#with ocean heat storage perturbation term set to 0

#Note: in some cases runs completed in the background but output did not print because 
#window had been closed. 

In [2]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd #necessary for new dimension in concat

In [3]:
#Perturbation version of the EBM with forcing, feedback, ocean heat uptake, and control temperature inputs
# 5-21-19: modified to only diffuse anomalous MSE rather than absolute. 

# Inputs:
# Rf: radiative forcing as a function of latitude
# lamda: feedback as a function of latitude (misspelled because "lambda" has special meaning in Python)
# G: ocean heat uptake (i.e. change in net surface heat fluxes) as a function of latitude
# T_control: temperature from the control climate (do things based on perturbations ot this)
# lats: latitudes for the model grid
# latBounds: can be specified, or automatically calculated based on given lats (put in default based on TRACMIP climatology common grid)
# anom: whether to diffuse anomalous rather than absolute MSE (papers and code from Aaron suggest it should be this way)
#
# Inputs with defalt values:
# D: Diffusion coefficient (m^2/s)
# dt: time step (arbitrary units)
# ps: surface pressure (Pa)
# RH: relative humidity (fraction)
# C: ocean heat capacity (arbitrary units)
# nmax: maximum number of iterations before giving up if equilibrium not reached
# tol: Max. temperature difference between 2 time steps to define equilibrium (K)
# tempPrint: if true, print the temperatures at selected latitudes when indicating progress
#
# Output: 2 XArray datasets, one on latitude and one on boundary latitude grid
def moistEBM_perturb(Rf, lamda, G, T_control, lats, latBounds='auto',
                  D=1.06e6, dt=1.0e-4, ps=9.8e4, RH=0.8, C=1, nmax=5e5, tol=0.001, tempPrint = False):
    
    #Physical constants
    Cp = 1004.   #Heat capacity of air at constant pressure (J/(kg*K))
    Lv = 2.5e6  #Latent heat vaporization at 0 deg C (J/kg)
    g = 9.8 #Gravitational acceleration (m/s^2)
    r_earth = 6.37e6 #Earth radius (m)
    Rd = 287. #Gas constant for dry air
    Rv = 461. #Gas constant for water vapor
    Lv = 2.5e6 #Latent heat of vaporization
    
    TS = T_control+0 #Initial temperature from control climate
    
    #Set up matrices for numerical integration of diffusion PDE
    #A and B represent transport of heat into grid box from the north and the south, respectively;
    #together they represent a 2nd-order finite-difference second derivative, weighted by latitude. 
    A = np.zeros((len(lats),len(lats)))
    B = np.zeros((len(lats),len(lats)))
    for i in range(1,len(lats)-1):
        A[i,i]=-1
        A[i,i+1]=1
        B[i,i]=1
        B[i,i-1]=-1
        
        
    #Calculate latitude boundaries if not specified
    if latBounds == 'auto':
        latBounds = np.append(-90, lats+(lats[1]-lats[0])/2.)
        
    #Define grid for EBM
    #print(latBounds)
    latDiff = np.diff(latBounds) #Grid spacing 
    latsNB = latBounds[1:len(latBounds)] #Latitudes at North Boundaries of grid boxes
    latsSB = latBounds[0:len(latBounds)-1] #Latitudes at South Boundaries of grid boxes
    lats_rad = lats*np.pi/180. #Convert from degrees to radians
    latsNB_rad = latsNB*np.pi/180.
    latsSB_rad = latsSB*np.pi/180.
    latDiff_rad = latDiff*np.pi/180.
    
    if tempPrint:
        #Print the temperatures at a few selected latitudes while running
        #Floor division to get integer indices
        printIndices=[len(lats)//10, 
                      len(lats)//5, 
                      len(lats)//3, 
                      len(lats)//2, 
                      (2*len(lats))//3, 
                      (4*len(lats))//5, 
                      (9*len(lats))//10]
        print('Latitudes for test temperature printing:')
        for k in range(len(printIndices)):
            print(lats[printIndices[k]])
    
    #Iterate through temperature tendency
    n = 0 #Iterative loop counter
    conv = False #Whether we've reached equilibrium
    while((n < nmax) & (conv == False)):
        #Set up profile of moist static energy
        #Saturation vapor pressure as a function of latitude:        
        es = 611.*np.exp((Lv/Rv)*((1./273.15)-(1./TS))) #Hartmann, 2016, eq. 1.11
        q = (RH*es)/ps*Rd/Rv #Specific humidity (kg/kg): RH = e/es therefore e = RH*es, and q = e/ps*Rd/Rv
        MSE = Cp*TS+Lv*q #Moist static energy at surface
        
        #Calculate climatological MSE and then perturbation by subtracting it out
        es_control = 611.*np.exp((Lv/Rv)*((1./273.15)-(1./T_control)))
        q_control = (RH*es_control)/ps*Rd/Rv
        MSE_control = Cp*T_control+Lv*q_control
        MSE_anom = MSE - MSE_control
         
        #Calculate diffusion/transport of anomalous MSE
        Transport = (D*(ps/g)/(r_earth*r_earth*np.cos(lats_rad)*latDiff_rad*latDiff_rad))*(
                    np.cos(latsNB_rad)*np.dot(A,MSE_anom)-np.cos(latsSB_rad)*np.dot(B,MSE_anom))
        #Diffusion at south boundary
        Transport[0] = (D*(ps/g)/(r_earth*r_earth*np.cos(lats_rad[0])*latDiff_rad[0]*latDiff_rad[0]))*(
                       np.cos(lats_rad[1])*(MSE_anom[2]-MSE_anom[1])-np.cos(lats_rad[0])*(MSE_anom[1]-MSE_anom[0]))
        #Diffusion at north boundary
        Transport[len(lats)-1] = (D*(ps/g)/(r_earth*r_earth*np.cos(lats_rad[len(lats)-1])*latDiff_rad[len(lats)-1]*latDiff_rad[len(lats)-1]))*(
                       np.cos(lats_rad[len(lats)-1])*(MSE_anom[len(lats)-1]-MSE_anom[len(lats)-2])-
                       np.cos(lats_rad[len(lats)-2])*(MSE_anom[len(lats)-2]-MSE_anom[len(lats)-3]))
        
        #Calculate perturbation temperature for feedback calculation (will be 0 on first time step)
        TS_perturb = TS - T_control
        
        #Temperature tendency/time stepping. When equilibrium is reached, the new and old TS cancel
        #and this equation represents energy balance, e.g. equation (1) of Bonan et al. (2018). 
        #Transport out
        TS = TS + dt*(Rf+lamda*TS_perturb+Transport-G)/C
        
        #Check for equilibrium (temperature has stopped changing) and return results if so
        #print(np.max(np.abs(EI-OLR+Transport)))
        if ((np.max(np.abs(Rf+lamda*TS_perturb+Transport-G)) < tol) and (n > 0)):
            conv = True
            print('EBM reached equilibrium in ' + str(n) + ' iterations')
            
            #Calculate the northward transport by integrating the transport/diffusion term northward
            NorthTransport = -np.cumsum(Transport*(2*np.pi*r_earth*r_earth)*(np.sin(latsNB_rad)-np.sin(latsSB_rad)))
            
            #Correct for transport at north pole (caused by numerics)
            N = NorthTransport[-1] #Northward energy transport at NP
            E = N/2.*(1+np.sin(lats*np.pi/180.)) 
            CorrectedTransport = NorthTransport - E
            
            #Versions with 0 appended (for plotting at boundary latitudes)
            NorthTransportBounds = np.append(0, NorthTransport)
            CorrectedTransportBounds = np.append(0, CorrectedTransport)
            
            #Return 2 XArray objects: ds_lat (for variables on the latitude grid) and ds_latBounds (for variables on boundary latitudes)
            ds_lat = xr.Dataset({'CorrectedTransport': (['lat'], CorrectedTransport),
                                 'Diffusion': (['lat'], Transport), 
                                 'MSE': (['lat'], MSE), 
                                 'MSE_anom': (['lat'], MSE_anom),
                                 'NorthTransport': (['lat'], NorthTransport), 
                                 'q': (['lat'], q), 
                                 'TS_perturb': ('lat', TS_perturb),
                                 'TS': (['lat'], TS)}, 
                                coords = {'lat': (['lat'], lats)})
            
#             ds_latBounds = xr.Dataset({'CorrectedTransportBounds': (['lat'], CorrectedTransportBounds), 
#                                        'NorthTransportBounds': (['lat'], NorthTransportBounds)}, 
#                                       coords ={'lat': (['lat'], latBounds)})
            
            return ds_lat
            
        if np.mod(n,6000)==0:
            print('EBM (perturbation) finished iteration ' + str(n))
            if tempPrint:
                print('{:e}, {:e}, {:e}, {:e}, {:e}, {:e}, {:e} K'.format(TS[printIndices[0]], 
                                                                          TS[printIndices[1]],
                                                                          TS[printIndices[2]],
                                                                          TS[printIndices[3]],
                                                                          TS[printIndices[4]],
                                                                          TS[printIndices[5]],
                                                                          TS[printIndices[6]]))
                
        n = n + 1 #Iterate loop counter
        #print(n)
        #Warning message if equilibrium not reached (Will return nothing)
        if(n == nmax):
            #print('WARNING: equilibrium not reached in moist EBM--nothing returned')
            print('WARNING: equilibrium not reached in moist EBM--nans returned')
            
            #Should really return all nans instead
            nanarray = np.ones(len(lats))*np.nan
            ds_lat = xr.Dataset({'CorrectedTransport': (['lat'], nanarray),
                                 'Diffusion': (['lat'], nanarray), 
                                 'MSE': (['lat'], nanarray), 
                                 'MSE_anom': (['lat'], nanarray),
                                 'NorthTransport': (['lat'], nanarray), 
                                 'q': (['lat'], nanarray), 
                                 'TS_perturb': ('lat', nanarray),
                                 'TS': (['lat'], nanarray)}, 
                                coords = {'lat': (['lat'], lats)})
            
            print(ds_lat)
            return ds_lat

In [4]:
#Load climatology data (for climatological temperature) and the Gregory regression output
ds_clim = xr.open_dataset('../nc_revised_20181130/master.nc')
ds_greg = xr.open_dataset('nc_from_xarray/GregoryTotalForcingFeedbackStorage_a4.nc')

In [5]:
#Lists of models

models = ['AM2', 'CAM3', 'CAM4', 'CNRM-AM6-DIA-v2', 'ECHAM-6.1', 'ECHAM-6.3',
       'IPSL-CM5A', 'MIROC5', 'MPAS', 'MetUM-GA6-CTL', 'MetUM-GA6-ENT',
       'NorESM2']

models_noCNRM = ['AM2', 'CAM3', 'CAM4', 'ECHAM-6.1', 'ECHAM-6.3',
       'IPSL-CM5A', 'MIROC5', 'MPAS', 'MetUM-GA6-CTL', 'MetUM-GA6-ENT',
       'NorESM2']

models_10 = ['AM2', 'CAM3', 'CAM4', 'ECHAM-6.1', 'ECHAM-6.3',
       'IPSL-CM5A', 'MIROC5', 'MPAS', 'MetUM-GA6-CTL', 'MetUM-GA6-ENT']

In [6]:
#Latitudes
lats = ds_clim.lat.data

In [7]:
#Load Gregory results and duplicate edge lons where necessary
ds_SW = xr.open_dataset('nc_from_xarray/GregoryAPRP_a4.nc')
ds_LW_ta = xr.open_dataset('nc_from_xarray/Gregory_kernel_ta_a4.nc')
ds_LW_wv = xr.open_dataset('nc_from_xarray/Gregory_kernel_wv_a4.nc')
ds_LW_ts = xr.open_dataset('nc_from_xarray/Gregory_kernel_ts_a4.nc')
ds_LW_cloud = xr.open_dataset('nc_from_xarray/Gregory_dLWCRE_co_a4.nc')

da_adj_SW_cloud = ds_SW['adj_cloud']
da_adj_SW_noncloud = ds_SW['adj_noncloud']
da_adj_LW_ta = -ds_LW_ta['adj']
da_adj_LW_wv = -ds_LW_wv['adj']
da_adj_LW_cloud = ds_LW_cloud['adj']

da_fb_SW_cloud = ds_SW['fb_cloud']
da_fb_SW_noncloud = ds_SW['fb_noncloud']
da_fb_LW_ta = -ds_LW_ta['fb']
da_fb_LW_wv = -ds_LW_wv['fb']
da_fb_LW_ts = -ds_LW_ts['fb'] #No rapid adjustment for this one
da_fb_LW_cloud = ds_LW_cloud['fb']

da_fb_LW_ts.loc[dict(lat=-89.5)] = da_fb_LW_ts.loc[dict(lat=-88.5)]
da_fb_LW_ts.loc[dict(lat=89.5)] = da_fb_LW_ts.loc[dict(lat=88.5)]

for model in models_10:
    if model in ['CAM3']:
        da_adj_LW_cloud.loc[dict(model=model, lat=-89.5)] = da_adj_LW_cloud.sel(model=model, lat=-87.5)
        da_adj_LW_cloud.loc[dict(model=model, lat=-88.5)] = da_adj_LW_cloud.sel(model=model, lat=-87.5)
        da_adj_LW_cloud.loc[dict(model=model, lat=89.5)] = da_adj_LW_cloud.sel(model=model, lat=87.5)
        da_adj_LW_cloud.loc[dict(model=model, lat=88.5)] = da_adj_LW_cloud.sel(model=model, lat=87.5)
        da_fb_LW_cloud.loc[dict(model=model, lat=-89.5)] = da_fb_LW_cloud.sel(model=model, lat=-87.5)
        da_fb_LW_cloud.loc[dict(model=model, lat=-88.5)] = da_fb_LW_cloud.sel(model=model, lat=-87.5)
        da_fb_LW_cloud.loc[dict(model=model, lat=89.5)] = da_fb_LW_cloud.sel(model=model, lat=87.5)
        da_fb_LW_cloud.loc[dict(model=model, lat=88.5)] = da_fb_LW_cloud.sel(model=model, lat=87.5)
    else:
        da_adj_LW_cloud.loc[dict(model=model, lat=-89.5)] = da_adj_LW_cloud.sel(model=model, lat=-88.5)
        da_adj_LW_cloud.loc[dict(model=model, lat=89.5)] = da_adj_LW_cloud.sel(model=model, lat=88.5)
        da_fb_LW_cloud.loc[dict(model=model, lat=-89.5)] = da_fb_LW_cloud.sel(model=model, lat=-88.5)
        da_fb_LW_cloud.loc[dict(model=model, lat=89.5)] = da_fb_LW_cloud.sel(model=model, lat=88.5)
        

In [51]:
#Calculate the instantaneous rapid adjustment (as residual) 
f_inst = ds_greg['forcing'] - (ds_SW['adj_cloud'] + ds_SW['adj_noncloud']
                                + ds_LW_cloud['adj'] - ds_LW_wv['adj'] - ds_LW_ta['adj'])

In [8]:
#Wrapper Functions to run EBM, specifying perturbations only at certain latitudes
#Actually don't need a new argument--just set the "scale" elsewhere to be 1 where we include the perturbation
#and 0 where we don't. 

# ds_clim: for climatological temperature
# ds_control: Dataset containing forcing, feedback, ocean heat storage terms
# da_perturb: DataArray loaded above for the rapid adjustments/feedbacks
# D: diffusivity
# dt, nmax: time step, max number of iterations
# Can enter dt, nmax as dicts, different for each model, instead.
# Try-except since don't want to break backward compatibility
# scale: percentage of the perturbation to apply (default 1 for backward compatibility)
#
# Returns: EBM_results_da: XArray DataArray containing the results for each model. 
# (save to NetCDF separately)

def run_EBM_perturb_adj(ds_clim, ds_control, da_perturb, models, D, dt=9.0e-5, nmax=7e5, scale=1, tempPrint=True):
    EBM_results_dict = dict()
    for model in models: 
        print(model)
        try:
            dt_model = dt[model] #input dict
        except:
            dt_model = dt
        try: 
            nmax_model = nmax[model]
        except: 
            nmax_model = nmax
        EBM_results_dict[model] = moistEBM_perturb(
            ds_control['forcing'].sel(model=model).data - da_perturb.sel(model=model).data * scale, 
            ds_control['feedback'].sel(model=model).data,
            0,
            ds_clim['ts'].sel(model=model, exp='AquaControl').mean(dim='lon').mean(dim='time').data,
            ds_clim.lat.data, 
            D=D, dt=dt_model, nmax=nmax_model, tempPrint=tempPrint
            )
    EBM_results_da = xr.concat(list(EBM_results_dict.values())[:], dim=pd.Index(models, name='model'))
    return EBM_results_da
    
def run_EBM_perturb_fb(ds_clim, ds_control, da_perturb, models, D, dt=9.0e-5, nmax=7e5, scale=1, tempPrint=True):
    EBM_results_dict = dict()
    for model in models:
        print(model)
        try:
            dt_model = dt[model] #input dict
        except:
            dt_model = dt
        try: 
            nmax_model = nmax[model]
        except: 
            nmax_model = nmax
        EBM_results_dict[model] = moistEBM_perturb(
            ds_control['forcing'].sel(model=model).data, 
            ds_control['feedback'].sel(model=model).data - da_perturb.sel(model=model).data * scale,
            0,
            ds_clim['ts'].sel(model=model, exp='AquaControl').mean(dim='lon').mean(dim='time').data,
            ds_clim.lat.data, 
            D=D, dt=dt_model, nmax=nmax_model, tempPrint=tempPrint
            )
    EBM_results_da = xr.concat(list(EBM_results_dict.values())[:], dim=pd.Index(models, name='model'))
    return EBM_results_da

In [9]:
#Define "scale" factors for tropical (within 30) and extratropical (poleward of 30) contributions
scale_tropics = np.ones(len(lats))
scale_tropics[np.abs(lats) > 30] = 0
print(scale_tropics)

scale_et = np.ones(len(lats))
scale_et[np.abs(lats) < 30] = 0
print(scale_et)

#All right, that's pretty simple. 

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 

In [10]:
print(lats)

[-89.5 -88.5 -87.5 -86.5 -85.5 -84.5 -83.5 -82.5 -81.5 -80.5 -79.5 -78.5
 -77.5 -76.5 -75.5 -74.5 -73.5 -72.5 -71.5 -70.5 -69.5 -68.5 -67.5 -66.5
 -65.5 -64.5 -63.5 -62.5 -61.5 -60.5 -59.5 -58.5 -57.5 -56.5 -55.5 -54.5
 -53.5 -52.5 -51.5 -50.5 -49.5 -48.5 -47.5 -46.5 -45.5 -44.5 -43.5 -42.5
 -41.5 -40.5 -39.5 -38.5 -37.5 -36.5 -35.5 -34.5 -33.5 -32.5 -31.5 -30.5
 -29.5 -28.5 -27.5 -26.5 -25.5 -24.5 -23.5 -22.5 -21.5 -20.5 -19.5 -18.5
 -17.5 -16.5 -15.5 -14.5 -13.5 -12.5 -11.5 -10.5  -9.5  -8.5  -7.5  -6.5
  -5.5  -4.5  -3.5  -2.5  -1.5  -0.5   0.5   1.5   2.5   3.5   4.5   5.5
   6.5   7.5   8.5   9.5  10.5  11.5  12.5  13.5  14.5  15.5  16.5  17.5
  18.5  19.5  20.5  21.5  22.5  23.5  24.5  25.5  26.5  27.5  28.5  29.5
  30.5  31.5  32.5  33.5  34.5  35.5  36.5  37.5  38.5  39.5  40.5  41.5
  42.5  43.5  44.5  45.5  46.5  47.5  48.5  49.5  50.5  51.5  52.5  53.5
  54.5  55.5  56.5  57.5  58.5  59.5  60.5  61.5  62.5  63.5  64.5  65.5
  66.5  67.5  68.5  69.5  70.5  71.5  72.5  73.5  7

In [11]:

#####   EBM RUNS   #####


In [12]:
#Tropical and extratropical versions of each of the experiments in EBM_Gregory_run_noG


In [ ]:
#Instantaneous CO2 forcing

In [53]:
EBM_results_f_inst_tr = run_EBM_perturb_adj(ds_clim, ds_greg, f_inst, models_10, D=0.96e6, scale=scale_tropics)

AM2
Latitudes for test temperature printing:
-71.5
-53.5
-29.5
0.5
30.5
54.5
72.5
EBM (perturbation) finished iteration 0
2.841682e+02, 2.896707e+02, 3.004598e+02, 3.080435e+02, 3.055918e+02, 2.953129e+02, 2.908846e+02 K
EBM (perturbation) finished iteration 6000
2.858733e+02, 2.912006e+02, 3.013335e+02, 3.085732e+02, 3.065073e+02, 2.968481e+02, 2.924424e+02 K
EBM (perturbation) finished iteration 12000
2.870239e+02, 2.921498e+02, 3.019565e+02, 3.090130e+02, 3.070661e+02, 2.977622e+02, 2.935972e+02 K
EBM (perturbation) finished iteration 18000
2.878020e+02, 2.927956e+02, 3.023846e+02, 3.093158e+02, 3.074443e+02, 2.983681e+02, 2.943531e+02 K
EBM (perturbation) finished iteration 24000
2.883287e+02, 2.932335e+02, 3.026763e+02, 3.095223e+02, 3.076998e+02, 2.987726e+02, 2.948543e+02 K
EBM (perturbation) finished iteration 30000
2.886843e+02, 2.935296e+02, 3.028742e+02, 3.096625e+02, 3.078723e+02, 2.990437e+02, 2.951886e+02 K
EBM (perturbation) finished iteration 36000
2.889242e+02, 2.93729

In [54]:
EBM_results_f_inst_et = run_EBM_perturb_adj(ds_clim, ds_greg, f_inst, models_10, D=0.96e6, scale=scale_et)

AM2
Latitudes for test temperature printing:
-71.5
-53.5
-29.5
0.5
30.5
54.5
72.5
EBM (perturbation) finished iteration 0
2.841678e+02, 2.896704e+02, 3.004603e+02, 3.080440e+02, 3.055914e+02, 2.953126e+02, 2.908842e+02 K
EBM (perturbation) finished iteration 6000
2.851240e+02, 2.910986e+02, 3.021024e+02, 3.094918e+02, 3.069038e+02, 2.965025e+02, 2.915709e+02 K
EBM (perturbation) finished iteration 12000
2.866842e+02, 2.924007e+02, 3.029549e+02, 3.100846e+02, 3.076282e+02, 2.976525e+02, 2.929980e+02 K
EBM (perturbation) finished iteration 18000
2.877689e+02, 2.932809e+02, 3.035180e+02, 3.104774e+02, 3.081197e+02, 2.984518e+02, 2.940077e+02 K
EBM (perturbation) finished iteration 24000
2.884941e+02, 2.938714e+02, 3.038997e+02, 3.107462e+02, 3.084557e+02, 2.989937e+02, 2.946887e+02 K
EBM (perturbation) finished iteration 30000
2.889803e+02, 2.942689e+02, 3.041585e+02, 3.109296e+02, 3.086843e+02, 2.993598e+02, 2.951464e+02 K
EBM (perturbation) finished iteration 36000
2.893074e+02, 2.94536

In [55]:
EBM_results_f_inst_tr.to_netcdf('nc_from_xarray/EBM_results_noG_f_inst_tr.nc')
EBM_results_f_inst_et.to_netcdf('nc_from_xarray/EBM_results_noG_f_inst_et.nc')

In [13]:
EBM_results_adj_SW_cloud_tr = run_EBM_perturb_adj(ds_clim, ds_greg, da_adj_SW_cloud, models, D=0.96e6, scale=scale_tropics)

AM2
Latitudes for test temperature printing:
-71.5
-53.5
-29.5
0.5
30.5
54.5
72.5
EBM (perturbation) finished iteration 0
2.841682e+02, 2.896707e+02, 3.004602e+02, 3.080441e+02, 3.055918e+02, 2.953129e+02, 2.908846e+02 K
EBM (perturbation) finished iteration 6000
2.867606e+02, 2.923250e+02, 3.027052e+02, 3.099289e+02, 3.076910e+02, 2.979536e+02, 2.934329e+02 K
EBM (perturbation) finished iteration 12000
2.892598e+02, 2.944053e+02, 3.040887e+02, 3.109165e+02, 3.089236e+02, 2.999245e+02, 2.958943e+02 K
EBM (perturbation) finished iteration 18000
2.909750e+02, 2.958208e+02, 3.050293e+02, 3.115897e+02, 3.097553e+02, 3.012313e+02, 2.975104e+02 K
EBM (perturbation) finished iteration 24000
2.921228e+02, 2.967734e+02, 3.056700e+02, 3.120512e+02, 3.103174e+02, 3.020956e+02, 2.985654e+02 K
EBM (perturbation) finished iteration 30000
2.928894e+02, 2.974125e+02, 3.061037e+02, 3.123650e+02, 3.106961e+02, 3.026698e+02, 2.992603e+02 K
EBM (perturbation) finished iteration 36000
2.934013e+02, 2.97840

In [14]:
EBM_results_adj_SW_cloud_et = run_EBM_perturb_adj(ds_clim, ds_greg, da_adj_SW_cloud, models, D=0.96e6, scale=scale_et)

AM2
Latitudes for test temperature printing:
-71.5
-53.5
-29.5
0.5
30.5
54.5
72.5
EBM (perturbation) finished iteration 0
2.841687e+02, 2.896709e+02, 3.004603e+02, 3.080440e+02, 3.055918e+02, 2.953131e+02, 2.908852e+02 K
EBM (perturbation) finished iteration 6000
2.881529e+02, 2.932099e+02, 3.030824e+02, 3.100427e+02, 3.078658e+02, 2.986311e+02, 2.947992e+02 K
EBM (perturbation) finished iteration 12000
2.911960e+02, 2.957478e+02, 3.047722e+02, 3.112470e+02, 3.093596e+02, 3.009778e+02, 2.976633e+02 K
EBM (perturbation) finished iteration 18000
2.932198e+02, 2.974448e+02, 3.059228e+02, 3.120785e+02, 3.103741e+02, 3.025246e+02, 2.995190e+02 K
EBM (perturbation) finished iteration 24000
2.945570e+02, 2.985753e+02, 3.067027e+02, 3.126480e+02, 3.110592e+02, 3.035447e+02, 3.007261e+02 K
EBM (perturbation) finished iteration 30000
2.954420e+02, 2.993278e+02, 3.072278e+02, 3.130343e+02, 3.115195e+02, 3.042197e+02, 3.015182e+02 K
EBM (perturbation) finished iteration 36000
2.960287e+02, 2.99828

In [15]:
EBM_results_adj_SW_cloud_tr.to_netcdf('nc_from_xarray/EBM_results_noG_adj_sw_cloud_tr.nc')
EBM_results_adj_SW_cloud_et.to_netcdf('nc_from_xarray/EBM_results_noG_adj_sw_cloud_et.nc')

In [16]:
EBM_results_adj_SW_noncloud_tr = run_EBM_perturb_adj(ds_clim, ds_greg, da_adj_SW_noncloud, models, D=0.96e6, scale=scale_tropics)

AM2
Latitudes for test temperature printing:
-71.5
-53.5
-29.5
0.5
30.5
54.5
72.5
EBM (perturbation) finished iteration 0
2.841682e+02, 2.896707e+02, 3.004602e+02, 3.080439e+02, 3.055918e+02, 2.953129e+02, 2.908846e+02 K
EBM (perturbation) finished iteration 6000
2.868358e+02, 2.924052e+02, 3.027811e+02, 3.098570e+02, 3.075857e+02, 2.978454e+02, 2.933299e+02 K
EBM (perturbation) finished iteration 12000
2.893586e+02, 2.944939e+02, 3.041573e+02, 3.108321e+02, 3.087901e+02, 2.997588e+02, 2.957116e+02 K
EBM (perturbation) finished iteration 18000
2.910637e+02, 2.958980e+02, 3.050866e+02, 3.114970e+02, 3.096086e+02, 3.010424e+02, 2.972977e+02 K
EBM (perturbation) finished iteration 24000
2.921963e+02, 2.968373e+02, 3.057175e+02, 3.119527e+02, 3.101639e+02, 3.018967e+02, 2.983409e+02 K
EBM (perturbation) finished iteration 30000
2.929503e+02, 2.974657e+02, 3.061439e+02, 3.122627e+02, 3.105388e+02, 3.024659e+02, 2.990304e+02 K
EBM (perturbation) finished iteration 36000
2.934530e+02, 2.97886

In [17]:
EBM_results_adj_SW_noncloud_et = run_EBM_perturb_adj(ds_clim, ds_greg, da_adj_SW_noncloud, models, D=0.96e6, scale=scale_et)

AM2
Latitudes for test temperature printing:
-71.5
-53.5
-29.5
0.5
30.5
54.5
72.5
EBM (perturbation) finished iteration 0
2.841682e+02, 2.896706e+02, 3.004603e+02, 3.080440e+02, 3.055918e+02, 2.953128e+02, 2.908845e+02 K
EBM (perturbation) finished iteration 6000
2.866267e+02, 2.922654e+02, 3.027471e+02, 3.098741e+02, 3.075750e+02, 2.977445e+02, 2.931510e+02 K
EBM (perturbation) finished iteration 12000
2.890822e+02, 2.943005e+02, 3.040880e+02, 3.108248e+02, 3.087510e+02, 2.996176e+02, 2.954872e+02 K
EBM (perturbation) finished iteration 18000
2.907522e+02, 2.956731e+02, 3.049931e+02, 3.114714e+02, 3.095490e+02, 3.008751e+02, 2.970465e+02 K
EBM (perturbation) finished iteration 24000
2.918640e+02, 2.965928e+02, 3.056080e+02, 3.119147e+02, 3.100904e+02, 3.017121e+02, 2.980723e+02 K
EBM (perturbation) finished iteration 30000
2.926050e+02, 2.972089e+02, 3.060240e+02, 3.122163e+02, 3.104560e+02, 3.022701e+02, 2.987506e+02 K
EBM (perturbation) finished iteration 36000
2.930998e+02, 2.97621

In [18]:
EBM_results_adj_SW_noncloud_tr.to_netcdf('nc_from_xarray/EBM_results_noG_adj_sw_noncloud_tr.nc')
EBM_results_adj_SW_noncloud_et.to_netcdf('nc_from_xarray/EBM_results_noG_adj_sw_noncloud_et.nc')

In [19]:
EBM_results_adj_LW_ta_tr = run_EBM_perturb_adj(ds_clim, ds_greg, da_adj_LW_ta, models, D=0.96e6, scale=scale_tropics)

AM2
Latitudes for test temperature printing:
-71.5
-53.5
-29.5
0.5
30.5
54.5
72.5
EBM (perturbation) finished iteration 0
2.841682e+02, 2.896707e+02, 3.004601e+02, 3.080439e+02, 3.055918e+02, 2.953129e+02, 2.908846e+02 K
EBM (perturbation) finished iteration 6000
2.867940e+02, 2.923539e+02, 3.027207e+02, 3.098054e+02, 3.075254e+02, 2.977874e+02, 2.932768e+02 K
EBM (perturbation) finished iteration 12000
2.892579e+02, 2.943936e+02, 3.040637e+02, 3.107554e+02, 3.086987e+02, 2.996509e+02, 2.955960e+02 K
EBM (perturbation) finished iteration 18000
2.909218e+02, 2.957638e+02, 3.049702e+02, 3.114030e+02, 3.094964e+02, 3.009027e+02, 2.971433e+02 K
EBM (perturbation) finished iteration 24000
2.920271e+02, 2.966803e+02, 3.055853e+02, 3.118468e+02, 3.100378e+02, 3.017367e+02, 2.981624e+02 K
EBM (perturbation) finished iteration 30000
2.927630e+02, 2.972936e+02, 3.060011e+02, 3.121487e+02, 3.104034e+02, 3.022927e+02, 2.988365e+02 K
EBM (perturbation) finished iteration 36000
2.932539e+02, 2.97704

In [20]:
EBM_results_adj_LW_ta_et = run_EBM_perturb_adj(ds_clim, ds_greg, da_adj_LW_ta, models, D=0.96e6, scale=scale_et)

AM2
Latitudes for test temperature printing:
-71.5
-53.5
-29.5
0.5
30.5
54.5
72.5
EBM (perturbation) finished iteration 0
2.841682e+02, 2.896705e+02, 3.004603e+02, 3.080440e+02, 3.055918e+02, 2.953128e+02, 2.908846e+02 K
EBM (perturbation) finished iteration 6000
2.863645e+02, 2.919904e+02, 3.025976e+02, 3.098233e+02, 3.075441e+02, 2.977213e+02, 2.932324e+02 K
EBM (perturbation) finished iteration 12000
2.886570e+02, 2.939047e+02, 3.038675e+02, 3.107336e+02, 3.086856e+02, 2.995540e+02, 2.955182e+02 K
EBM (perturbation) finished iteration 18000
2.902471e+02, 2.952150e+02, 3.047306e+02, 3.113515e+02, 3.094533e+02, 3.007687e+02, 2.970220e+02 K
EBM (perturbation) finished iteration 24000
2.913160e+02, 2.960998e+02, 3.053194e+02, 3.117748e+02, 3.099717e+02, 3.015722e+02, 2.980045e+02 K
EBM (perturbation) finished iteration 30000
2.920321e+02, 2.966947e+02, 3.057184e+02, 3.120627e+02, 3.103210e+02, 3.021063e+02, 2.986522e+02 K
EBM (perturbation) finished iteration 36000
2.925113e+02, 2.97093

In [21]:
EBM_results_adj_LW_ta_tr.to_netcdf('nc_from_xarray/EBM_results_noG_adj_lw_ta_tr.nc')
EBM_results_adj_LW_ta_et.to_netcdf('nc_from_xarray/EBM_results_noG_adj_lw_ta_et.nc')

In [22]:
EBM_results_adj_LW_wv_tr = run_EBM_perturb_adj(ds_clim, ds_greg, da_adj_LW_wv, models_noCNRM, D=0.96e6, scale=scale_tropics)

AM2
Latitudes for test temperature printing:
-71.5
-53.5
-29.5
0.5
30.5
54.5
72.5
EBM (perturbation) finished iteration 0
2.841682e+02, 2.896707e+02, 3.004603e+02, 3.080439e+02, 3.055918e+02, 2.953129e+02, 2.908846e+02 K
EBM (perturbation) finished iteration 6000
2.870640e+02, 2.926816e+02, 3.030988e+02, 3.100815e+02, 3.077330e+02, 2.979754e+02, 2.934413e+02 K
EBM (perturbation) finished iteration 12000
2.898724e+02, 2.949983e+02, 3.046170e+02, 3.111544e+02, 3.090495e+02, 3.000540e+02, 2.960214e+02 K
EBM (perturbation) finished iteration 18000
2.917520e+02, 2.965431e+02, 3.056381e+02, 3.118867e+02, 3.099484e+02, 3.014572e+02, 2.977518e+02 K
EBM (perturbation) finished iteration 24000
2.929920e+02, 2.975711e+02, 3.063299e+02, 3.123892e+02, 3.105598e+02, 3.023930e+02, 2.988916e+02 K
EBM (perturbation) finished iteration 30000
2.938141e+02, 2.982568e+02, 3.067968e+02, 3.127312e+02, 3.109728e+02, 3.030166e+02, 2.996447e+02 K
EBM (perturbation) finished iteration 36000
2.943608e+02, 2.98714

In [23]:
EBM_results_adj_LW_wv_et = run_EBM_perturb_adj(ds_clim, ds_greg, da_adj_LW_wv, models_noCNRM, D=0.96e6, scale=scale_et)

AM2
Latitudes for test temperature printing:
-71.5
-53.5
-29.5
0.5
30.5
54.5
72.5
EBM (perturbation) finished iteration 0
2.841682e+02, 2.896707e+02, 3.004603e+02, 3.080440e+02, 3.055918e+02, 2.953128e+02, 2.908845e+02 K
EBM (perturbation) finished iteration 6000
2.868079e+02, 2.924231e+02, 3.028318e+02, 3.099090e+02, 3.076118e+02, 2.977845e+02, 2.931683e+02 K
EBM (perturbation) finished iteration 12000
2.893633e+02, 2.945355e+02, 3.042203e+02, 3.108892e+02, 3.088172e+02, 2.996978e+02, 2.955532e+02 K
EBM (perturbation) finished iteration 18000
2.910856e+02, 2.959510e+02, 3.051551e+02, 3.115569e+02, 3.096383e+02, 3.009885e+02, 2.971532e+02 K
EBM (perturbation) finished iteration 24000
2.922270e+02, 2.968961e+02, 3.057891e+02, 3.120148e+02, 3.101964e+02, 3.018497e+02, 2.982083e+02 K
EBM (perturbation) finished iteration 30000
2.929861e+02, 2.975280e+02, 3.062176e+02, 3.123264e+02, 3.105736e+02, 3.024242e+02, 2.989066e+02 K
EBM (perturbation) finished iteration 36000
2.934921e+02, 2.97950

In [24]:
EBM_results_adj_LW_wv_tr.to_netcdf('nc_from_xarray/EBM_results_noG_adj_lw_wv_tr.nc')
EBM_results_adj_LW_wv_et.to_netcdf('nc_from_xarray/EBM_results_noG_adj_lw_wv_et.nc')

In [25]:
EBM_results_adj_LW_cloud_tr = run_EBM_perturb_adj(ds_clim, ds_greg, da_adj_LW_cloud, models_10, D=0.96e6, scale=scale_tropics)

AM2
Latitudes for test temperature printing:
-71.5
-53.5
-29.5
0.5
30.5
54.5
72.5
EBM (perturbation) finished iteration 0
2.841682e+02, 2.896707e+02, 3.004603e+02, 3.080436e+02, 3.055918e+02, 2.953129e+02, 2.908846e+02 K
EBM (perturbation) finished iteration 6000
2.869360e+02, 2.925250e+02, 3.029172e+02, 3.099143e+02, 3.076613e+02, 2.979166e+02, 2.933943e+02 K
EBM (perturbation) finished iteration 12000
2.895774e+02, 2.947085e+02, 3.043526e+02, 3.109321e+02, 3.089135e+02, 2.999018e+02, 2.958634e+02 K
EBM (perturbation) finished iteration 18000
2.913555e+02, 2.961716e+02, 3.053206e+02, 3.116265e+02, 3.097658e+02, 3.012359e+02, 2.975105e+02 K
EBM (perturbation) finished iteration 24000
2.925334e+02, 2.971483e+02, 3.059771e+02, 3.121025e+02, 3.103445e+02, 3.021241e+02, 2.985938e+02 K
EBM (perturbation) finished iteration 30000
2.933161e+02, 2.978009e+02, 3.064205e+02, 3.124263e+02, 3.107351e+02, 3.027157e+02, 2.993095e+02 K
EBM (perturbation) finished iteration 36000
2.938374e+02, 2.98237

In [ ]:
EBM_results_adj_LW_cloud_et = run_EBM_perturb_adj(ds_clim, ds_greg, da_adj_LW_cloud, models_10, D=0.96e6, scale=scale_et)

AM2
Latitudes for test temperature printing:
-71.5
-53.5
-29.5
0.5
30.5
54.5
72.5
EBM (perturbation) finished iteration 0
2.841681e+02, 2.896706e+02, 3.004603e+02, 3.080440e+02, 3.055918e+02, 2.953128e+02, 2.908844e+02 K
EBM (perturbation) finished iteration 6000
2.863234e+02, 2.920895e+02, 3.026888e+02, 3.098462e+02, 3.075266e+02, 2.975694e+02, 2.928242e+02 K
EBM (perturbation) finished iteration 12000
2.886662e+02, 2.940298e+02, 3.039658e+02, 3.107507e+02, 3.086436e+02, 2.993517e+02, 2.950547e+02 K
EBM (perturbation) finished iteration 18000
2.902679e+02, 2.953407e+02, 3.048255e+02, 3.113634e+02, 3.094012e+02, 3.005534e+02, 2.965546e+02 K
EBM (perturbation) finished iteration 24000
2.913355e+02, 2.962198e+02, 3.054097e+02, 3.117835e+02, 3.099157e+02, 3.013550e+02, 2.975440e+02 K
EBM (perturbation) finished iteration 30000
2.920480e+02, 2.968093e+02, 3.058052e+02, 3.120696e+02, 3.102635e+02, 3.018902e+02, 2.981994e+02 K
EBM (perturbation) finished iteration 36000
2.925243e+02, 2.97204

In [ ]:
EBM_results_adj_LW_cloud_tr.to_netcdf('nc_from_xarray/EBM_results_noG_adj_lw_cloud_tr.nc')
EBM_results_adj_LW_cloud_et.to_netcdf('nc_from_xarray/EBM_results_noG_adj_lw_cloud_et.nc')

In [ ]:

### Feedback Runs ###


In [ ]:
EBM_results_fb_SW_cloud_tr = run_EBM_perturb_fb(ds_clim, ds_greg, da_fb_SW_cloud, models, D=0.96e6, scale=scale_tropics)

In [ ]:
EBM_results_fb_SW_cloud_et = run_EBM_perturb_fb(ds_clim, ds_greg, da_fb_SW_cloud, models, D=0.96e6, scale=scale_et)

In [ ]:
EBM_results_fb_SW_cloud_tr.to_netcdf('nc_from_xarray/EBM_results_noG_fb_sw_cloud_tr.nc')
EBM_results_fb_SW_cloud_et.to_netcdf('nc_from_xarray/EBM_results_noG_fb_sw_cloud_et.nc')

In [ ]:
EBM_results_fb_SW_noncloud_tr = run_EBM_perturb_fb(ds_clim, ds_greg, da_fb_SW_noncloud, models, D=0.96e6, scale=scale_tropics)

In [ ]:
EBM_results_fb_SW_noncloud_et = run_EBM_perturb_fb(ds_clim, ds_greg, da_fb_SW_noncloud, models, D=0.96e6, scale=scale_et)

In [ ]:
EBM_results_fb_SW_noncloud_tr.to_netcdf('nc_from_xarray/EBM_results_noG_fb_sw_noncloud_tr.nc')
EBM_results_fb_SW_noncloud_et.to_netcdf('nc_from_xarray/EBM_results_noG_fb_sw_noncloud_et.nc')

In [ ]:
EBM_results_fb_LW_cloud_tr = run_EBM_perturb_fb(ds_clim, ds_greg, da_fb_LW_cloud, models_10, D = 0.96e6, scale=scale_tropics)

In [ ]:
EBM_results_fb_LW_cloud_et = run_EBM_perturb_fb(ds_clim, ds_greg, da_fb_LW_cloud, models_10, D = 0.96e6, scale=scale_et)

In [ ]:
EBM_results_fb_LW_cloud_tr.to_netcdf('nc_from_xarray/EBM_results_noG_fb_lw_cloud_tr.nc')
EBM_results_fb_LW_cloud_et.to_netcdf('nc_from_xarray/EBM_results_noG_fb_lw_cloud_et.nc')

In [ ]:
EBM_results_fb_LW_wv_tr = run_EBM_perturb_fb(ds_clim, ds_greg, da_fb_LW_wv, models_noCNRM, D=0.96e6, scale=scale_tropics)

In [ ]:
EBM_results_fb_LW_wv_et = run_EBM_perturb_fb(ds_clim, ds_greg, da_fb_LW_wv, models_noCNRM, D=0.96e6, scale=scale_et)

In [ ]:
EBM_results_fb_LW_wv_tr.to_netcdf('nc_from_xarray/EBM_results_noG_fb_lw_wv_tr.nc')
EBM_results_fb_LW_wv_et.to_netcdf('nc_from_xarray/EBM_results_noG_fb_lw_wv_et.nc')

In [ ]:
EBM_results_fb_LW_ts_tr = run_EBM_perturb_fb(ds_clim, ds_greg, da_fb_LW_ts, models, D=0.96e6, dt=5.0e-5, nmax=1e6, scale=scale_tropics)

In [ ]:
EBM_results_fb_LW_ts_et = run_EBM_perturb_fb(ds_clim, ds_greg, da_fb_LW_ts, models, D=0.96e6, dt=5.0e-5, nmax=1e6, scale=scale_et)

In [ ]:
EBM_results_fb_LW_ts_tr.to_netcdf('nc_from_xarray/EBM_results_noG_fb_lw_ts_tr.nc')
EBM_results_fb_LW_ts_et.to_netcdf('nc_from_xarray/EBM_results_noG_fb_lw_ts_et.nc')

In [ ]:
#                                       #
# Partial temperature perturbation runs #
#                                       #

#Already have a scale factor of .05, .1, .15; multiply this by scale_tropics or scale_et

In [ ]:
dt_dict_LW_ta_05p = {'AM2':             2.0e-5, 
                     'CAM3':            9.0e-5, 
                     'CAM4':            9.0e-5, 
                     'CNRM-AM6-DIA-v2': 9.0e-5, 
                     'ECHAM-6.1':       9.0e-5, 
                     'ECHAM-6.3':       9.0e-5, 
                     'IPSL-CM5A':       9.0e-5, 
                     'MIROC5':          9.0e-5, 
                     'MPAS':            9.0e-5, 
                     'MetUM-GA6-CTL':   9.0e-5, 
                     'MetUM-GA6-ENT':   9.0e-5, 
                     'NorESM2':         9.0e-5}

EBM_results_fb_LW_ta_05p_tr = run_EBM_perturb_fb(ds_clim, ds_greg, da_fb_LW_ta, models, 
                                                       D=0.96e6, scale=.05*scale_tropics, dt=dt_dict_LW_ta_05p, nmax=2e6)

EBM_results_fb_LW_ta_05p_et = run_EBM_perturb_fb(ds_clim, ds_greg, da_fb_LW_ta, models, 
                                                       D=0.96e6, scale=.05*scale_et, dt=dt_dict_LW_ta_05p, nmax=2e6)


In [ ]:
EBM_results_fb_LW_ta_05p_tr.to_netcdf('nc_from_xarray/EBM_results_noG_fb_lw_ta_05p_tr.nc')
EBM_results_fb_LW_ta_05p_et.to_netcdf('nc_from_xarray/EBM_results_noG_fb_lw_ta_05p_et.nc')

In [ ]:
dt_dict_LW_ta_10p = {'AM2':             2.0e-5, 
                     'CAM3':            9.0e-5, 
                     'CAM4':            9.0e-5, 
                     'CNRM-AM6-DIA-v2': 9.0e-5, 
                     'ECHAM-6.1':       2.0e-5, 
                     'ECHAM-6.3':       2.0e-5, 
                     'IPSL-CM5A':       9.0e-5, 
                     'MIROC5':          9.0e-5, 
                     'MPAS':            9.0e-5, 
                     'MetUM-GA6-CTL':   2.0e-5, 
                     'MetUM-GA6-ENT':   2.0e-5, 
                     'NorESM2':         9.0e-5}

EBM_results_fb_LW_ta_10p_tr = run_EBM_perturb_fb(ds_clim, ds_greg, da_fb_LW_ta, models, 
                                              D=0.96e6, scale=.1*scale_tropics, dt=dt_dict_LW_ta_10p, nmax=2e6)

EBM_results_fb_LW_ta_10p_et = run_EBM_perturb_fb(ds_clim, ds_greg, da_fb_LW_ta, models, 
                                              D=0.96e6, scale=.1*scale_et, dt=dt_dict_LW_ta_10p, nmax=2e6)

In [ ]:
EBM_results_fb_LW_ta_10p_tr.to_netcdf('nc_from_xarray/EBM_results_noG_fb_lw_ta_10p_tr.nc')
EBM_results_fb_LW_ta_10p_et.to_netcdf('nc_from_xarray/EBM_results_noG_fb_lw_ta_10p_et.nc')

In [ ]:
dt_dict_LW_ta_15p = {'AM2':             2.3e-5, 
                     'CAM3':            9.0e-5, 
                     'CAM4':            9.0e-5, 
                     'CNRM-AM6-DIA-v2': 2.3e-5, 
                     'ECHAM-6.1':       2.3e-5, 
                     'ECHAM-6.3':       2.3e-5, 
                     'IPSL-CM5A':       2.3e-5, 
                     'MIROC5':          9.0e-5, 
                     'MPAS':            9.0e-5, 
                     'MetUM-GA6-CTL':   2.0e-5, 
                     'MetUM-GA6-ENT':   2.0e-5, 
                     'NorESM2':         9.0e-5}

EBM_results_fb_LW_ta_15p_tr = run_EBM_perturb_fb(ds_clim, ds_greg, da_fb_LW_ta, models, 
                                              D=0.96e6, scale=.15*scale_tropics, dt=dt_dict_LW_ta_15p, nmax=3e6)

EBM_results_fb_LW_ta_15p_et = run_EBM_perturb_fb(ds_clim, ds_greg, da_fb_LW_ta, models, 
                                              D=0.96e6, scale=.15*scale_et, dt=dt_dict_LW_ta_15p, nmax=3e6)

In [ ]:
EBM_results_fb_LW_ta_15p_tr.to_netcdf('nc_from_xarray/EBM_results_noG_fb_lw_ta_15p_tr.nc')
EBM_results_fb_LW_ta_15p_et.to_netcdf('nc_from_xarray/EBM_results_noG_fb_lw_ta_15p_et.nc')

In [ ]:
#Server was disconnected while this was running, preventing all the output from showing up. However, 
#NetCDF files show it made it at least as far as the 10% ta case. 